In [25]:
# %% High-FPS pick & place with contact-gated weld (headless, Jupyter)
import math, threading
import numpy as np
from IPython.display import display
import ipywidgets as W

import pybullet as p
import pybullet_data as pd
import cv2

# -------------------- Config --------------------
SIM_FPS     = 240
TARGET_FPS  = 30               # try 24–30
WIDTH,HEIGHT= 384,288          # lower => faster
USE_EGL     = False            # set True if EGL is available on your node

# -------------------- Fresh sim --------------------
try: p.disconnect()
except: pass
if USE_EGL:
    p.connect(p.DIRECT, options="--egl")
else:
    p.connect(p.DIRECT)

p.resetSimulation()
p.setAdditionalSearchPath(pd.getDataPath())
p.setGravity(0,0,-9.81)
p.setTimeStep(1.0/SIM_FPS)

p.loadURDF("plane.urdf")
p.loadURDF("table/table.urdf", [0.5,0,-0.625], useFixedBase=True)
panda = p.loadURDF("franka_panda/panda.urdf", useFixedBase=True)

# Cube sized to fit gripper; placed on table
CUBE_SCALE = 1.3
cube_xyz = [0.60, 0.00, 0.02 + 0.12]   # (x,y,z) ~ on table top
cube  = p.loadURDF("cube_small.urdf", cube_xyz, globalScaling=CUBE_SCALE, useFixedBase=False)

EE_LINK, FINGER_L, FINGER_R = 11, 9, 10
renderer = p.ER_BULLET_HARDWARE_OPENGL if USE_EGL else p.ER_TINY_RENDERER

# -------------------- Widgets --------------------
yaw   = W.FloatSlider(description="Yaw",   min=-180, max=180, step=1, value=50)
pitch = W.FloatSlider(description="Pitch", min=-89,  max=0,   step=1, value=-30)
dist  = W.FloatSlider(description="Dist",  min=0.4,  max=1.8, step=0.05, value=0.9)
tx    = W.FloatSlider(description="TargetX", min=0.3, max=0.8, step=0.01, value=0.55)
ty    = W.FloatSlider(description="TargetY", min=-0.3,max=0.3, step=0.01, value=0.00)
tz    = W.FloatSlider(description="TargetZ", min=0.05,max=0.60,step=0.01, value=0.18)
grip  = W.FloatSlider(description="Gripper", min=0.0, max=0.08, step=0.001, value=0.08)

start_btn = W.Button(description="▶ Start", button_style="success")
stop_btn  = W.Button(description="■ Stop",  button_style="danger")
respawn_btn = W.Button(description="↻ Respawn cube", tooltip="Remove and spawn cube", icon="refresh")


img = W.Image(format='jpeg')
img.layout.border = '1px solid #444'
img.layout.width  = f"{WIDTH}px"
img.layout.height = f"{HEIGHT}px"

ui = W.VBox([
    W.HTML("<b>Camera</b>"),
    W.HBox([yaw, pitch, dist]),
    W.HTML("<b>Camera Target</b>"),
    W.HBox([tx, ty, tz]),
    W.HTML("<b>Robot</b>"),
    W.HBox([grip, start_btn, stop_btn, respawn_btn]),
    img
])
display(ui)

# -------------------- Respawn handler --------------------
def on_respawn(_):
    if stop_flag["run"]:
        print("Stop the run before respawning the cube.")
        return
    global cube, cube_xyz
    try:
        p.removeBody(cube)
    except Exception:
        pass
    # Respawn at default (with small random XY jitter on the table)
    x = float(np.clip(0.60 + np.random.uniform(-0.05, 0.05), 0.45, 0.75))
    y = float(np.clip(0.00 + np.random.uniform(-0.05, 0.05), -0.20, 0.20))
    table_top_z = 0.12   # matches your initial placement
    cube_xyz = [x, y, 0.02 + table_top_z]
    cube = p.loadURDF("cube_small.urdf", cube_xyz, globalScaling=CUBE_SCALE, useFixedBase=False)
    # Show an updated frame
    img.value = grab_frame_bytes()

respawn_btn.on_click(on_respawn)

# -------------------- Helpers --------------------
def set_gripper(width, force=220):
    half = float(np.clip(width/2, 0.0, 0.04))
    p.setJointMotorControl2(panda, FINGER_L, p.POSITION_CONTROL, targetPosition=half, force=force)
    p.setJointMotorControl2(panda, FINGER_R, p.POSITION_CONTROL, targetPosition=half, force=force)

def set_arm_target(xyz, rpy):
    orn = p.getQuaternionFromEuler(rpy)
    q = p.calculateInverseKinematics(
        panda, EE_LINK, xyz, orn,
        lowerLimits=[-2.9]*7, upperLimits=[2.9]*7, jointRanges=[5.8]*7,
        restPoses=[0]*7, residualThreshold=1e-4, maxNumIterations=200
    )
    for j in range(7):
        p.setJointMotorControl2(panda, j, p.POSITION_CONTROL, targetPosition=q[j], force=800)

def contact_both_fingers_with(bodyB):
    c1 = p.getContactPoints(bodyA=panda, bodyB=bodyB, linkIndexA=FINGER_L)
    c2 = p.getContactPoints(bodyA=panda, bodyB=bodyB, linkIndexA=FINGER_R)
    return (len(c1) > 0) and (len(c2) > 0)

def weld_to_hand(bodyB):
    # fixed constraint at current relative transform hand<->bodyB
    hand = p.getLinkState(panda, EE_LINK, computeForwardKinematics=True)
    hand_pos, hand_orn = hand[4], hand[5]
    obj_pos, obj_orn   = p.getBasePositionAndOrientation(bodyB)
    inv_pos, inv_orn   = p.invertTransform(hand_pos, hand_orn)
    parent_pos, parent_orn = p.multiplyTransforms(inv_pos, inv_orn, obj_pos, obj_orn)
    cid = p.createConstraint(
        parentBodyUniqueId=panda, parentLinkIndex=EE_LINK,
        childBodyUniqueId=bodyB, childLinkIndex=-1,
        jointType=p.JOINT_FIXED, jointAxis=[0,0,0],
        parentFramePosition=parent_pos, parentFrameOrientation=parent_orn,
        childFramePosition=[0,0,0], childFrameOrientation=[0,0,0,1]
    )
    p.changeConstraint(cid, maxForce=250)
    return cid

def grab_frame_bytes():
    view = p.computeViewMatrixFromYawPitchRoll(
        cameraTargetPosition=[tx.value, ty.value, tz.value],
        distance=dist.value, yaw=yaw.value, pitch=pitch.value, roll=0, upAxisIndex=2)
    proj = p.computeProjectionMatrixFOV(60, WIDTH/HEIGHT, 0.01, 2.5)
    w,h,rgba,_,_ = p.getCameraImage(WIDTH, HEIGHT, view, proj,
                                    renderer=renderer,
                                    flags=p.ER_NO_SEGMENTATION_MASK)[:5]
    rgba = np.reshape(rgba, (h, w, 4))
    bgr = cv2.cvtColor(rgba[...,:3], cv2.COLOR_RGB2BGR)
    ok, buf = cv2.imencode(".jpg", bgr, [cv2.IMWRITE_JPEG_QUALITY, 70])
    return buf.tobytes() if ok else None

# -------------------- Main streaming loop --------------------
stop_flag = {"run": False}

def main_loop():
    stop_flag["run"] = True
    down = (math.pi, 0, 0)
    steps_per_frame = max(1, int(SIM_FPS / TARGET_FPS))
    t_mod = 0
    weld_id = None

    # sequence waypoints
    home  = [0.50, 0.00, 0.50]
    above = [cube_xyz[0], cube_xyz[1], 0.30]
    grasp = [cube_xyz[0], cube_xyz[1], 0.12]
    place = [0.55, -0.15, 0.30]

    # helper: drive to target for N steps, streaming frames
    def drive(xyz, rpy, steps, g=None):
        nonlocal t_mod
        if g is not None: set_gripper(g)
        set_arm_target(xyz, rpy)
        for s in range(steps):
            if not stop_flag["run"]: return False
            p.stepSimulation()
            if (t_mod % steps_per_frame) == 0:
                frame = grab_frame_bytes()
                if frame is not None: img.value = frame
            t_mod += 1
        return True

    # open hand and go home
    set_gripper(grip.value)
    if not drive(home,  down, 240): return
    if not drive(above, down, 160): return

    # descend, close, create weld if both fingers contact
    if not drive(grasp, down, 120): return
    set_gripper(0.028)
    # wait a bit to register contacts
    if not drive(grasp, down, 60): return
    if contact_both_fingers_with(cube):
        weld_id = weld_to_hand(cube)

    # lift & move to place
    if not drive(above, down, 160, g=0.028): return
    if not drive(place, down, 200, g=0.028): return

    # release
    if weld_id is not None:
        p.removeConstraint(weld_id)
    set_gripper(0.08)
    if not drive(place, down, 80): return

    # back home
    drive(home, down, 200)
    stop_flag["run"] = False

def on_start(_):
    if not stop_flag["run"]:
        threading.Thread(target=main_loop, daemon=True).start()

def on_stop(_):
    stop_flag["run"] = False

start_btn.on_click(on_start)
stop_btn.on_click(on_stop)
respawn_btn.on_click(on_respawn)

# show one initial frame
img.value = grab_frame_bytes()
